In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support import expected_conditions as Expect
import time

# Step 1: Get the website

In [2]:
browser = webdriver.Chrome(executable_path='/Users/zhangfurong/Desktop/coding demo/chromedriver')
browser.get('https://www.ringdata.com/news/result?keyWords=&startTime=2005-01-01&endTime=2020-12-31&position=2&param=%7B%22allKeyWords%22%3A%22%E8%87%AA%E6%9D%80%22,%22anyKeyWords%22%3A%22%E9%AB%98%E6%A0%A1%E5%AD%A6%E7%94%9F%20%E5%A4%A7%E4%B8%80%E5%AD%A6%E7%94%9F%20%E5%A4%A7%E4%BA%8C%E5%AD%A6%E7%94%9F%20%E5%A4%A7%E4%B8%89%E5%AD%A6%E7%94%9F%20%E5%A4%A7%E5%9B%9B%E5%AD%A6%E7%94%9F%20%E7%A0%94%E4%B8%80%E5%AD%A6%E7%94%9F%20%E7%A0%94%E4%BA%8C%E5%AD%A6%E7%94%9F%20%E7%A0%94%E4%B8%89%E5%AD%A6%E7%94%9F%20%E5%BA%94%E5%B1%8A%E6%AF%95%E4%B8%9A%E7%94%9F%20%E5%A4%A7%E5%AD%A6%E7%94%9F%20%E7%A0%94%E7%A9%B6%E7%94%9F%20%E5%8D%9A%E5%A3%AB%E7%94%9F%20%E6%A0%A1%E5%86%85%22,%22notKeyWords%22%3A%22%E6%9E%AA%E6%9D%80%22,%22selectTime%22%3A%22custom%22,%22position%22%3A%222%22,%22website%22%3A%22%22,%22startTime%22%3A%222005-01-01%22,%22endTime%22%3A%222020-12-31%22%7D')

# Step 2: Log in the website

In [28]:
from selenium.webdriver.common.by import By
# click the log_in buttom
browser.find_element_by_xpath('//*[@id="app"]/section/header/div/div[3]/span[2]/i').click() 
# wait for log_in page
Wait(browser, 10).until(Expect.text_to_be_present_in_element((By.XPATH,"//*[@id='app']/section/footer/div/div"),u'Copyright'))
# log in 
browser.find_element_by_xpath('//*[@id="app"]/section/main/div[2]/form/div[1]/div/div[1]/input').send_keys('18757579281')
browser.find_element_by_xpath('//*[@id="app"]/section/main/div[2]/form/div[2]/div/div[1]/input').send_keys('19971213zfr')
browser.find_element_by_xpath('//*[@id="app"]/section/main/div[2]/form/div[4]/div/div/button[1]/span').click()

# Step 3: Data Crawling

In [29]:
# define the variables
titles = []
abstracts = []
sourceTimes = []
urls = []

# find the whole page xpath
results = browser.find_elements_by_xpath("//*[@id='app']/section/main/div/div[2]/div[3]/div")


In [30]:
#find the elements xpath which we want to get
for i in range(42):  #the number in range means the amount of pages
    for result in results: #this loop means getting each news
        titleOne = result.find_element_by_xpath(".//h3/a")
        abstractOne = result.find_element_by_xpath(".//div[1]")
        sourceTimeOne  = result.find_element_by_xpath(".//div[2]")
        # add one row and get the text content
        titles.append(titleOne.text)
        abstracts.append(abstractOne.text)
        sourceTimes.append(sourceTimeOne.text)
        # click the title to get URLs
        element = result.find_element_by_xpath('.//h3/a') 
        browser.execute_script("arguments[0].click();", element) # click relatively news title
        handles = (browser.window_handles) 
        browser.switch_to.window(handles[1]) #switch to the specific news page
        urlOne = browser.current_url
        urls.append(urlOne)
        browser.close()
        browser.switch_to.window(handles[0])
        # this part reference:CSDN
    # click to next page
    element = browser.find_element_by_xpath('//*[@id="app"]/section/main/div/div[2]/div[4]/button[2]')
    browser.execute_script("arguments[0].click();", element) 
    Wait(browser, 5)

print(len(titles))
print(len(abstracts))
print(len(sourceTimes))
print(len(urls))


20
20
20
20


In [34]:
dataRearch = {
    'Title': titles,
    'Abstract': abstracts,
    'source&dates': sourceTimes,
    'Url':urls}
# print(dataRearch)

In [35]:
pddataRearch = pd.DataFrame.from_dict(dataRearch)
pddataRearch

,TITLE,Abstract,source&dates,Url
0,武汉理工研究生自杀 研究生为何会自杀,现在的学生压力很大，如果一旦承受不了，甚至会做出自杀的事情来，近日网曝武汉理工研究生自杀，而...,新淮安网 2018-04-01,https://www.ringdata.com/news/detail?id=201804...
1,美大学生在校内开枪扫射后自杀,胡乱开枪 枪击发生在得克萨斯大学奥斯汀分校。校方发言人朗达·韦尔登说，上午8时许，一名男...,燕赵晚报 2010-09-30,https://www.ringdata.com/news/detail?id=201009...
2,女研究生宿舍内自杀,9日，网上各大论坛热传几近相同的帖子，上海海事大学海商法女研究生杨元元在宿舍卫生间自杀身亡，...,滁州网 2009-12-14,https://www.ringdata.com/news/detail?id=200912...
3,女研究生跳运河自杀,20日上午8时30分许，北京通州区东关大桥北侧，北京物资学院一名研二女生被发现跳入运河轻...,呼和浩特日报 2011-02-22,https://www.ringdata.com/news/detail?id=201102...
4,论文“卡壳”，女研究生自杀,A05-A06版,华声在线网 2011-07-08,https://www.ringdata.com/news/detail?id=201107...
5,疑因论文未过研究生自杀,5月18日，28岁的中南大学研三学生姜东身坠楼身亡。 5月23日，姜东身哥哥姜东中告诉记...,中国常州网 2015-05-25,https://www.ringdata.com/news/detail?id=201505...
6,外经贸研究生自杀于校外,本报北京2月22日讯（记者田浩通讯员王丽英于洋）李女士正在对外经济贸易大学留校工作并就读研究...,人民法院报 2010-02-23,https://www.ringdata.com/news/detail?id=201002...
7,女研究生和网友相约自杀,女研究生和网友相约自杀 两人留遗书称自愿，一人死亡一人获救；家人称死者因病影响学业，心理压力...,潇湘晨报 2015-07-05,https://www.ringdata.com/news/detail?id=201507...
8,中南大学研究生留遗书自杀,据《苏州日报》消息 28岁的中南大学机电工程学院研三学生姜东身从学校图书馆六楼跳...,滁州网 2015-05-26,https://www.ringdata.com/news/detail?id=201505...
9,贫困女研究生 宿舍自杀身亡,国内新闻……7版,长春晚报 2009-12-11,https://www.ringdata.com/news/detail?id=200912...


In [36]:
pddataRearch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   TITLE         20 non-null     object
 1   Abstract      20 non-null     object
 2   source&dates  20 non-null     object
 3   Url           20 non-null     object
dtypes: object(4)
memory usage: 768.0+ bytes


In [ ]:
pddataRearch.to_csv('/Users/zhangfurong/Desktop/coding demo/Data.csv')